## Sobol Financial Examples

In this notebook we demonstrate methods for pricing 2 types of call option (European and Asian), using 2 techniques:
- Black Scholes
- Monte Carlo

The Black Scholes functions:
- `bsEuroCall`
- `bsAsiaCall`
give a closed-form solution, with a deterministic result. Their only dependence is the `cnorm` function from `NormalDistribution.q`.

The Monte Carlo functions:
- `mcEuroCall`
- `mcAsiaCall`
each perform a number (nPaths) of random experiments.

For each experiment, we:
1. Generate a number (nSteps) of random numbers (Uniform or Sobol)
2. Convert into N(0,1) random numbers (using `invcnorm`)
3. Convert into a Wiener path random walk (using Standard or Brownian Bridge method)
4. Convert into an asset price path, based on params (s,v,r,q,t)
5. Convert into an option price, based on option type and strike price (k)
Then average the prices to get a final predicted price.

For each option type, we generate:
- A single Black Scholes price.
- A number (nTrials) of Monte Carlo prices.
We can then take the RMSE of the Monte Carlo prices, versus the Black Scholes price, to get an error.

We repeat this, for increasing values of nPaths, and compare the resulting errors.

---

In [1]:
/ load in q scripts
\l ../code/MonteCarlo.q
\l ../code/BlackScholes.q

In [2]:
/ float variables
/ * s = Asset at t=0
/ * k = Strike
/ * v = Volatility
/ * r = Interest rate, if Asian > .1
/ * q = Dividents
/ * t = Expiry
params:`s`k`v`r`q`t`mkt!(100 100 .2 .05 0 1.),`

/ integer variables
nSamplesStart:8      / initial number of samples
nSamplesTotal:128    / total number of paths
nsteps :1000         / number of time steps (per sample path)
ntrials:20           / number of trials (for RMSE)

/ booleans
/ * sobol   = SobolSeq(1), Random(0)
/ * bbridge = BrownianBridge(1), Standard(0)
bools:([]sobol:110b;bbridge:101b)

In [3]:
mainfnc:{[p;nstart;ntot;nsteps;ntrials;bools]
 -1$[b:`asian~p`mkt;"Asian call";"European call"];t:.z.t;
 bs:$[b;bsAsiaCall nsteps;bsEuroCall]. params`s`k`v`r`q`t;
 -1"No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price";
 N:nstart;
 while[N<ntot;
  r:mcComparison[bs]each mcTrials[;params;nsteps;N;ntrials]each bools;
  -1" , "sv enlist[string N],string[r[;1]],enlist string last r[;0];
  N+:64];
 -1"CPU time = ",string[.z.t-t],"\n";}

In [4]:
peuro :params,enlist[`mkt]!enlist`european
pasian:params,enlist[`mkt]!enlist`asian
mainfnc[;nSamplesStart;nSamplesTotal;nsteps;ntrials;bools]each(peuro;pasian);

European call
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 10.45058 , 10.45058 , 10.45058 , 0
72 , 10.45058 , 10.45058 , 10.45058 , 0
CPU time = 00:00:45.414

Asian call
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 5.55152 , 5.55152 , 5.55152 , 0
72 , 5.55152 , 5.55152 , 5.55152 , 0
CPU time = 00:00:45.050



---